# EU Regional Economic Impact Model (2022)
This notebook builds the backend of a scalable economic impact model for the EU, using Eurostat's 2022 FIGARO supply and use tables and regional data. It closely follows the RHOMOLO V4 methodology developed by the Joint Research Centre.


## 1. Setup: Import Libraries and Define File Paths

In [9]:
import pandas as pd 
import numpy as np
import os
from typing import Dict, Tuple

# Define local data directories
data_dir = "C:/Users/dzsve/Dezernat Zukunft e.V/DZ-Schalte - Dokumente/Sven/IO Model/Data/2022"
previous_data_dir = "C:/Users/dzsve/Dezernat Zukunft e.V/DZ-Schalte - Dokumente/Sven/IO Model/Data/Previous years"

## 2. Supply and Use Tables (FIGARO)

### 2.1 Load and Inspect SUTs

In [10]:
supply_table = pd.read_csv(os.path.join(data_dir, "FIGARO_supply_2022.csv"), index_col=0)
use_table = pd.read_csv(os.path.join(data_dir, "FIGARO_use_2022.csv"), index_col=0)

supply_table.head()

,AR_A01,AR_A02,AR_A03,AR_B,AR_C10T12,AR_C13T15,AR_C16,AR_C17,AR_C18,AR_C19,...,ZA_P85,ZA_Q86,ZA_Q87_88,ZA_R90T92,ZA_R93,ZA_S94,ZA_S95,ZA_S96,ZA_T,ZA_U
rowLabels,,,,,,,,,,,,,,,,,,,,,
AR_CPA_A01,73118.056,59.295,0.000,0.000,4014.862,62.723,0.000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
AR_CPA_A02,0.000,921.066,0.000,0.000,2.048,0.000,20.145,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
AR_CPA_A03,0.000,0.000,2695.841,0.000,403.270,0.000,0.000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
AR_CPA_B,0.000,0.000,0.000,31549.851,39.558,0.000,0.000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
AR_CPA_C10T12,1713.321,0.000,104.449,0.000,105373.108,0.702,0.000,0.0,0.0,0.698,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


### 2.2 Correct Negative Values in SUTs

Following the RHOMOLO method, theoretically invalid negative values are replaced with zero and correction factors (stock adjustments) are added.

In [11]:
# Correct negative values in supply table
if(supply_table < 0).any().any():
    original_supply_table = supply_table.copy()
    supply_corrections = supply_table.copy()
    supply_table[supply_table < 0] = 0 #replace negatives with 0
    supply_corrections = (supply_corrections - supply_table) #store corrections
    
    # Add discrepancy row & column in the supply table
    supply_table.loc["Stock Adjustments", :] = supply_corrections.sum(axis = 0) #Column correction
    supply_table.loc[:, "Stock Adjustments"] = supply_corrections.sum(axis = 1) #Row correction
    supply_table.loc["Stock Adjustments", "Stock Adjustments"] = -supply_corrections.values.sum()

# Correct negative values in supply table
if(use_table < 0).any().any():
    original_use_table = use_table.copy()
    use_corrections = use_table.copy()
    use_table[use_table < 0] = 0 #replace negatives with 0
    use_corrections = (use_corrections - use_table) #store corrections
    
    # Add discrepancy row & column in the use table
    use_table.loc["Stock Adjustments", :] = use_corrections.sum(axis=0)  # Column correction
    use_table.loc[:, "Stock Adjustments"] = use_corrections.sum(axis=1)  # Row correction
    use_table.loc["Stock Adjustments", "Stock Adjustments"] = -use_corrections.values.sum()

## 3. Regional Data
The regional indicators used to map national SUTs to NUTS-2 level are loaded and cleaned.
### 3.1 Set up Regional Accounts and SBS Data
#### 3.1.1 Load Data

In [12]:
rows_to_drop = ["GEO (Labels)", "GEO (Codes)", "NACE_R2 (Labels)", "Extra-Regio NUTS 2", "ITM_NEWA (Labels)", "Special value", ":", "Observation flags:", "p", "e", "u", 
                "b", "m", "d", "be", "de", "bd", "confidential", "estimated", "not available", "low reliability", "break in time series"]

def load_excel(file_name, sheet="Sheet 1", skip_rows=8, index_col=0, drop_rows=rows_to_drop):
    """Loads and cleans an Excel file by dropping unwanted columns and empty rows."""
    df = pd.read_excel(os.path.join(data_dir, file_name), sheet_name=sheet, skiprows=skip_rows, index_col=index_col)
    df = df[~df.index.isin(drop_rows)] # Removes specific rows by index 
    df = df.dropna(how='all')
    df = df[df.columns.drop(list(df.filter(regex="Unnamed")))]
    df = df.replace(":", np.nan)
    return df

# Load regional data
GVA_2022 = load_excel("nama_10r_3gva__GVA by region industry.xlsx")
CoE_2022 = load_excel("nama_10r_2coe__COE by region industry.xlsx")
Employment_2022 = load_excel("nama_10r_3empers__Employed persons by region industry.xlsx", skip_rows=9)
GDPpc_2022 = load_excel("nama_10r_3gdp__GDP per inhabitant by region.xlsx", skip_rows=7)
HoursWorked_2022 = load_excel("nama_10r_2emhrw__Hours worked by region industry.xlsx", skip_rows=9)
PrimaryIncome_2022 = load_excel("nama_10r_2hhinc__Balance primary incomes by region.xlsx", skip_rows=9)
Agriculture_2022 = load_excel("agr_r_accts__Agri regional accounts.xlsx", skip_rows=9)


# Load Structural Business Statistics (SBS) datasets
SBSEmployment_2022 = load_excel("sbs_r_nuts2021__Persons employed by region industry.xlsx", index_col = 1)
SBSFirms_2022 = load_excel("sbs_r_nuts2021__Firms by region industry.xlsx", index_col = 1)
SBSWages_2022 = load_excel("sbs_r_nuts2021__Wages by region industry.xlsx", index_col = 1)

SBSEmployment_2022.drop(SBSEmployment_2022.tail(3).index,inplace=True)
SBSFirms_2022.drop(SBSFirms_2022.tail(3).index,inplace=True)
SBSWages_2022.drop(SBSWages_2022.tail(3).index,inplace=True)

SBSEmployment_2022 = SBSEmployment_2022.iloc[:, 1:]
SBSFirms_2022 = SBSFirms_2022.iloc[:, 1:]
SBSWages_2022 = SBSWages_2022.iloc[:, 1:]

c:\Users\dzsve\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\dzsve\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\dzsve\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\dzsve\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\dzsve\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains

#### 3.1.2 Standardise NUTS Codes as Row Names across all Dataframes

In [13]:
# Load NUTS-2 region mapping from an SBS file (e.g. persons employed)
NUTS_mapping = load_excel("sbs_r_nuts2021__Persons employed by region industry.xlsx", index_col = False)
NUTS_mapping = NUTS_mapping.iloc[2:289, :2]
NUTS_mapping.columns = ["NUTS_Code", "Region_Name"]

#remove rows that have have "Extra-Regio NUTS 2" as value in second column
NUTS_mapping = NUTS_mapping.loc[~NUTS_mapping["Region_Name"].str.contains("Extra-Regio NUTS 2")]

# Remove extra whitespace, standardise
NUTS_mapping["NUTS_Code"] = NUTS_mapping["NUTS_Code"].str.strip()
NUTS_mapping["Region_Name"] = NUTS_mapping["Region_Name"].str.strip()
NUTS_mapping["Country_Code"] = NUTS_mapping["NUTS_Code"].str[:2]

# Remove "00" from NUTS-2 codes
NUTS_mapping["NUTS_Code"] = NUTS_mapping["NUTS_Code"].str.replace("00", "", regex=False)

# Remove duplicates
NUTS_mapping = NUTS_mapping[~NUTS_mapping["NUTS_Code"].duplicated(keep='first')]

# Build a dictionary for NUTS-2 to NUTS-0 mapping
regionname_to_nuts = NUTS_mapping.set_index("Region_Name")["NUTS_Code"].to_dict()

#load official 2024 nuts codes
NUTS2024 = pd.read_excel(os.path.join(data_dir,"NUTS2021-NUTS2024.xlsx"), sheet_name="NUTS2024")
NUTS2024 = NUTS2024.iloc[:, 0:4]
NUTS2024 = NUTS2024[NUTS2024.iloc[:, 3] == 2]
NUTS2024 = NUTS2024.iloc[:, 0:3]
NUTS2024 = NUTS2024[~NUTS2024.iloc[:, 2].str.contains('Extra-Regio NUTS 2')]
NUTS2024.iloc[:, 1] = NUTS2024.iloc[:, 1].astype(str).str.replace("00$", "", regex=True)


def convert_index_to_nuts(df, mapping_dict):
    """
    Convert index from region names to NUTS codes using a dictionary and handle special region mappings.
    If no exact match for region name in mapping_dict, keep original index value.
    Handles special cases for NL31, NL33, PT16, PT17, PT18 regions.
    """
    df = df.copy()
    
    # First convert index using mapping dictionary
    df.index = [mapping_dict.get(idx, idx) for idx in df.index]
    
    # Define special region mappings
    special_mappings = {
        'NL31': 'NL35',
        'NL33': 'NL36', 
        'PT16': 'PT19',
        'PT17': 'PT1A',
        'PT18': 'PT1C'
    }
    
    # Process each special region
    for old_code, new_code in special_mappings.items():
        if old_code in df.index:
            # Check if row has any non-null values
            if df.loc[old_code].notna().any():
                # If new code doesn't exist, create it
                if new_code not in df.index:
                    df.loc[new_code] = df.loc[old_code]
                # If new code exists but has all null values, fill with old code values
                elif df.loc[new_code].isna().all():
                    df.loc[new_code] = df.loc[old_code]
                # If new code exists and has values, do nothing
            # Remove old code row
            df = df.drop(old_code)
        elif new_code not in df.index:
            # Create new code row if neither old nor new code exists
            df.loc[new_code] = pd.Series(index=df.columns, dtype=float)

    # Keep only rows where index is in either column 1 or column 2 of NUTS2024
    df = df[df.index.isin(NUTS2024.loc[:, "Country code"]) | df.index.isin(NUTS2024.loc[:, "NUTS Code"])]
    # Remove duplicate indices keeping first occurrence
    df = df[~df.index.duplicated(keep='first')]
    
    # Create rows for any missing NUTS codes
    missing_nuts = set(NUTS2024.loc[:, "NUTS Code"]) - set(df.index)
    for nuts_code in missing_nuts:
        df.loc[nuts_code] = pd.Series(index=df.columns, dtype=float)

    
    return df


c:\Users\dzsve\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:

# Store all dataframes in a dictionary
dfs = {
    "GVA_2022": GVA_2022,
    "CoE_2022": CoE_2022,
    "Employment_2022": Employment_2022,
    "GDPpc_2022": GDPpc_2022,
    "HoursWorked_2022": HoursWorked_2022,
    "PrimaryIncome_2022": PrimaryIncome_2022,
    "Agriculture_2022": Agriculture_2022,
    "SBSEmployment_2022": SBSEmployment_2022,
    "SBSFirms_2022": SBSFirms_2022,
    "SBSWages_2022": SBSWages_2022
}

# Apply conversion and cleaning to each dataframe
for name in dfs:
    dfs[name] = convert_index_to_nuts(dfs[name], regionname_to_nuts)

# Unpack the cleaned dataframes back into your working variables
(
    GVA_2022, CoE_2022, Employment_2022, GDPpc_2022, HoursWorked_2022,
    PrimaryIncome_2022, Agriculture_2022,
    SBSEmployment_2022, SBSFirms_2022, SBSWages_2022
) = dfs.values()



In [15]:
# Find indexes in SBSEmployment_2022 but not in Others
only_in_sbs_agr = SBSEmployment_2022.index.difference(Agriculture_2022.index)
only_in_sbs_coe = SBSEmployment_2022.index.difference(CoE_2022.index)
only_in_sbs_prim_inc = SBSEmployment_2022.index.difference(PrimaryIncome_2022.index)
only_in_sbs_hours = SBSEmployment_2022.index.difference(HoursWorked_2022.index)

print(only_in_sbs_agr)
print(only_in_sbs_coe)
print(only_in_sbs_hours)
print(only_in_sbs_prim_inc)

Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')


### 3.2 Clean Dataframes & Impute Missing Data
#### 3.2.1 Check for Missing Values in Regional Datasets
Verify if any values are missing in the SBS and regional accounts datasets.

In [16]:
# check for missing values in dataframes
def check_missing(df, name):
    missing = df.isnull().sum().sum()
    print(f"{name}: {'No missing values.' if missing == 0 else f'{missing} missing values.'}")
    return missing

missing_summary = {
    "GVA_2022": check_missing(GVA_2022, "GVA_2022"),
    "CoE_2022": check_missing(CoE_2022, "CoE_2022"),   
    "Employment_2022": check_missing(Employment_2022, "Employment_2022"),
    "GDPpc_2022": check_missing(GDPpc_2022, "GDPpc_2022"),
    "HoursWorked_2022": check_missing(HoursWorked_2022, "HoursWorked_2022"),
    "PrimaryIncome_2022": check_missing(PrimaryIncome_2022, "PrimaryIncome_2022"),
    "Agriculture_2022": check_missing(Agriculture_2022, "Agriculture_2022"),
    "SBSEmployment_2022": check_missing(SBSEmployment_2022, "SBSEmployment_2022"),
    "SBSFirms_2022": check_missing(SBSFirms_2022, "SBSFirms_2022"),
    "SBSWages_2022": check_missing(SBSWages_2022, "SBSWages_2022"),
}

GVA_2022: No missing values.
CoE_2022: 24 missing values.
Employment_2022: No missing values.
GDPpc_2022: No missing values.
HoursWorked_2022: 372 missing values.
PrimaryIncome_2022: 2 missing values.
Agriculture_2022: 330 missing values.
SBSEmployment_2022: 1691 missing values.
SBSFirms_2022: 833 missing values.
SBSWages_2022: 2425 missing values.


In [23]:
# Filter SBSEmployment_2022 to only include rows with 2-digit indices
SBSEmployment_2022_2digit = SBSEmployment_2022[SBSEmployment_2022.index.str.len() == 2]

# Print column names with missing values in SBSEmployment_2022_2digit
missing_columns = SBSEmployment_2022_2digit.columns[SBSEmployment_2022_2digit.isnull().any()].tolist()
print("Columns with missing values in SBSEmployment_2022_2digit:")
print(missing_columns)
print(f"Total number of columns with missing values: {len(missing_columns)}")


# Print all rows that have missing values in SBSEmployment_2022_2digit
missing_rows = SBSEmployment_2022_2digit[SBSEmployment_2022_2digit.isnull().any(axis=1)]
print("Rows with missing values in SBSEmployment_2022_2digit:")
print(missing_rows.index.tolist())
print(f"\nTotal number of rows with missing values: {len(missing_rows)}")



Columns with missing values in SBSEmployment_2022_2digit:
['B', 'B05', 'B06', 'B07', 'B08', 'B09', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C26', 'C27', 'C29', 'C30', 'C31', 'D', 'D35', 'E36', 'E37', 'E38', 'E39', 'G45', 'G47', 'H50', 'H51', 'H52', 'H53', 'I', 'I55', 'J59', 'J60', 'J61', 'J62', 'J63', 'K64', 'K65', 'K66', 'M69', 'M70', 'M72', 'M74', 'N77', 'N78', 'N79', 'N80', 'Q', 'Q86', 'R', 'R90', 'R91', 'R92', 'R93']
Total number of columns with missing values: 62
Rows with missing values in SBSEmployment_2022_2digit:
['CZ', 'EE', 'IE', 'FR', 'IT', 'CY', 'LT', 'LU', 'HU', 'MT', 'AT', 'PL', 'PT', 'SI', 'SK']

Total number of rows with missing values: 15


In [32]:
# Create dataframe as matrix of GDP per capita differences between all regions
gdp_pc_matrix = pd.DataFrame(index=GDPpc_2022.index, columns=GDPpc_2022.index)

for i in GDPpc_2022.index:
    for j in GDPpc_2022.index:
        gdp_pc_matrix.loc[i, j] = abs(GDPpc_2022.loc[i].iloc[0] - GDPpc_2022.loc[j].iloc[0])



Closest regions to CZ (diff=200.00): ['ES11', 'ES13']
Closest regions to EE (diff=0.00): ['ES41']
Closest regions to IE (diff=8100.00): ['DK01']
Closest regions to FR (diff=0.00): ['DEB1']
Closest regions to IT (diff=100.00): ['FRI1', 'PL91']
Closest regions to CY (diff=100.00): ['BE35']
Closest regions to LT (diff=0.00): ['CZ02']
Closest regions to LU (diff=5000.00): ['IE05']
Closest regions to HU (diff=100.00): ['PL']
Closest regions to MT (diff=0.00): ['DEG0', 'FRJ2']
Closest regions to AT (diff=100.00): ['DE23']
Closest regions to PL (diff=100.00): ['EL65', 'HU']
Closest regions to PT (diff=500.00): ['CZ02', 'LT']
Closest regions to SI (diff=0.00): ['EL30']
Closest regions to SK (diff=100.00): ['PT20']


In [ ]:
for country in missing_rows.index:
    # Get all regions except the country itself
    other_regions = gdp_pc_matrix.loc[country].drop(country)
    
    # Find the minimum difference
    min_diff = other_regions.min()
    
    # Find all regions with the minimum difference
    closest_regions = other_regions[other_regions == min_diff].index.tolist()
    
    print(f"Closest regions to {country} (diff={min_diff:.2f}): {closest_regions}")

### 3.2.2 Compute Missing Data in SBS Dataframes

The imputation strategy uses GDP per capita similarity to match regions for filling missing data, with a preference for regions within the same country (25% weight reduction for cross-country matches). This approach ensures that missing values are imputed using data from economically similar regions, while still allowing cross-country matches when necessary. The imputed values are then scaled to ensure that the sum of regional values matches the national totals, while preserving all original data.

Helper functions are used to apply this imputation strategy to all relevant datasets. The imputation strategy is described in detail below:

**SBS Data Imputation Strategy**
1. Similarity Calculation
    - GDP Per Capita Similarity
        - For each pair of regions:
            - Calculate absolute difference in GDP per capita
            - Convert to similarity score using 1/(1 + difference)
            - Apply country preference factor:
                - 4x higher weight for regions in same country
                - 0.25x weight for regions in different countries
        - Function: `prepare_similarity_matrix(gdp_pc, sbs_data)`

2. Country-Level Imputation (2-digit codes) <br>
    For each country and industry with missing data:
    1. Find similar regions:
      - Look at all regions with data for this industry
        - Use similarity matrix to find most similar regions
        - Calculate weights based on similarity scores
    2. Calculate relative structure:
        - For the country:
            - Get available industries
            - Calculate proportions of each industry
        - For similar regions:
            - Calculate their industry proportions
            - Take weighted average based on similarity
    - Function: `calculate_relative_structure(df, region, available_industries)` <br>

    3. Impute missing value:
        - If country has other industries:
            - Use relative structure to estimate proportion
            - Scale by total of available industries
        - If no other industries:
            - Use weighted average of similar regions' values
    - Function: `impute_missing_sbs_data(sbs_data, similarity_matrix)`

3. Allocation Calculation <br>
    For each country and industry:
    1. Get country total:
        - Use country-level value (original or imputed)
    2. Calculate regional sum:
        - Sum all regional values (4-digit codes)
    3. Calculate allocation:
        - Allocation = Country Total - Regional Sum
        - Positive: need to distribute more to regions
        - Negative: regions sum to more than country total
    - Function: `calculate_missing_allocation(df, country, industry)`

4. Regional Imputation (4-digit codes) <br>
    For each country and industry with missing regional data:
    1. Find missing regions:
        - Identify all regional rows with missing values
        - Find all regions with available data
    2. For each missing region:
        - Calculate similarity:
            - Find similar regions using similarity matrix
            - Calculate weights based on similarity scores
        - Calculate relative structure:
            - For the region:
                - Get available industries
                - Calculate proportions of each industry
            - For similar regions:
                - Calculate their industry proportions
                - Take weighted average based on similarity
        - Function: `calculate_relative_structure(df, region, available_industries)` <br>

        - Impute value:
            - If region has other industries:
                - Use relative structure to estimate proportion
                - Scale by allocation amount
            - If no other industries:
                - Use weighted average of similar regions' values
        - Function: `impute_missing_sbs_data(sbs_data, similarity_matrix)`
    3. Scale imputed values:
        - If allocation is positive:
            - Sum all imputed values
            - Scale to match allocation amount
        - If allocation is negative:
            - Keep imputed values as is
        - Function: `impute_missing_sbs_data(sbs_data, similarity_matrix)`

5. Convergence Check
    - Compare imputed values with previous iteration
        - If changes are below tolerance threshold, stop
        - Otherwise, repeat regional imputation
    - Function: `impute_missing_sbs_data(sbs_data, similarity_matrix)`

6. Process All Datasets
    - Apply the imputation process to all SBS datasets
    - Verify results against country totals
    - Function: `process_sbs_datasets(sbs_data, gdp_pc)`

    <br>

**Key Features**
1. Hierarchical Approach
    - Country-level data imputed first
    - Regional data imputed while respecting country totals
2. Relative Structure Preservation
    - Uses industry proportions to guide imputation
    - Maintains economic structure of regions
3. Similarity Matching
    - Uses GDP per capita similarity
    - Gives preference to regions from same country
4. Consistency Checks
    - Ensures regional sums match country totals
    - Handles both positive and negative allocations
5. Iterative Refinement
    - Multiple iterations to improve imputation
    - Convergence check to ensure stability


This strategy ensures that:
- Country-level data is imputed first using similar regions
- Regional data is imputed while maintaining consistency with country totals
- The economic structure of regions is preserved
- The imputation is based on GDP per capita similarity

In [14]:

def prepare_similarity_matrix(
    gdp_pc: pd.DataFrame,
    sbs_data: pd.DataFrame
) -> pd.DataFrame:
    """
    Creates a similarity matrix based on GDP per capita differences between regions.
    
    Strategy:
    1. Calculate GDP-based similarity using inverse of differences
    2. Apply country preference factor (4x higher for same country)
    
    Args:
        gdp_pc (pd.DataFrame): GDP per capita data by region
        sbs_data (pd.DataFrame): SBS data with missing values (not used in this version)
        
    Returns:
        pd.DataFrame: GDP-based similarity matrix
    """
    print("\nPreparing similarity matrix...")
    
    # Initialize matrix
    similarity_matrix = pd.DataFrame(index=gdp_pc.index, columns=gdp_pc.index)
    
    # Calculate GDP-based similarity
    print("Calculating GDP-based similarities...")
    for r1 in gdp_pc.index:
        for r2 in gdp_pc.index:
            # Calculate absolute difference in GDP per capita
            diff = abs(gdp_pc.loc[r1, 'GDPpc_2022'] - gdp_pc.loc[r2, 'GDPpc_2022'])
            # Convert difference to similarity score
            gdp_score = 1 / (1 + diff)
            
            # Apply country preference factor
            if r1[:2] == r2[:2]:  # Same country
                similarity_matrix.loc[r1, r2] = gdp_score
            else:  # Different country
                similarity_matrix.loc[r1, r2] = gdp_score * 0.25
    
    print("Similarity matrix preparation complete")
    return similarity_matrix

def calculate_missing_allocation(
    df: pd.DataFrame,
    country: str,
    industry: str
) -> float:
    """
    Calculates the amount to be allocated for a specific industry in a country.
    
    Strategy:
    1. Get country-level total (2-digit code)
    2. Sum up all regional data (4-digit codes)
    3. Calculate difference (positive means we need to allocate more)
    
    Args:
        df (pd.DataFrame): Data with missing values
        country (str): Country code
        industry (str): Industry code
        
    Returns:
        float: Amount to be allocated for this industry
    """
    # Get country-level data and regional sum
    country_total = df.loc[country, industry]
    regional_sum = df[df.index.str.startswith(country) & 
                     (df.index.str.len() == 4)][industry].sum()
    
    # Calculate allocation needed
    allocation = country_total - regional_sum
    return allocation

def impute_missing_sbs_data(
    sbs_data: pd.DataFrame,
    similarity_matrix: pd.DataFrame,
    gdp_pc: pd.DataFrame = None,
    max_iterations: int = 100,
    tolerance: float = 1e-6
) -> pd.DataFrame:
    """
    Imputes missing SBS data using GDP per capita similarity.
    
    Strategy:
    1. Impute country-level data using similar regions
    2. Calculate missing allocations for each country and industry
    3. Impute regional data while respecting allocations
    
    Args:
        sbs_data (pd.DataFrame): SBS data with missing values
        similarity_matrix (pd.DataFrame): GDP-based similarity matrix
        gdp_pc (pd.DataFrame, optional): GDP per capita data. If None, will use existing similarity matrix
        max_iterations (int): Maximum number of iterations for convergence
        tolerance (float): Convergence tolerance
        
    Returns:
        pd.DataFrame: Imputed SBS data
    """
    print("\nStarting SBS data imputation...")
    
    # Create a copy to avoid modifying original data
    imputed_data = sbs_data.copy()
    
    # Convert values with ":" to NaN
    imputed_data = imputed_data.replace(":", np.nan)
    
    # If GDP per capita data is provided, recalculate similarity matrix
    if gdp_pc is not None:
        print("Recalculating similarity matrix with new GDP data...")
        similarity_matrix = prepare_similarity_matrix(gdp_pc, sbs_data)
    
    # Step 1: Impute country-level data
    print("\nStep 1: Imputing country-level data...")
    country_rows = imputed_data[imputed_data.index.str.len() == 2]
    for country in country_rows.index:
        for industry in country_rows.columns:
            if pd.isna(country_rows.loc[country, industry]):
                # Get similar regions with data for this industry
                available_regions = imputed_data.index[~imputed_data[industry].isna()]
                similar_regions = similarity_matrix.loc[country, available_regions]
                weights = similar_regions / similar_regions.sum()
                
                # Use weighted average of available values
                imputed_value = (imputed_data.loc[available_regions, industry] * weights).sum()
                imputed_data.loc[country, industry] = imputed_value
    
    # Step 2: Calculate missing allocations for each country and industry
    print("\nStep 2: Calculating missing allocations...")
    missing_allocations = {}
    for country in imputed_data.index.str[:2].unique():
        missing_allocations[country] = {}
        for industry in imputed_data.columns:
            missing_allocations[country][industry] = calculate_missing_allocation(
                imputed_data, country, industry
            )
    
    # Step 3: Impute regional data
    for iteration in range(max_iterations):
        # Store previous state for convergence check
        previous_data = imputed_data.copy()
        
        # Process each country separately
        for country in imputed_data.index.str[:2].unique():
            # Get data for current country
            country_mask = imputed_data.index.str.startswith(country)
            country_data = imputed_data[country_mask]
            
            # Process each industry
            for industry in country_data.columns:
                missing_mask = country_data[industry].isna()
                if missing_mask.any():
                    # Identify regions with missing and available data
                    missing_regions = country_data.index[missing_mask]
                    available_regions = imputed_data.index[~imputed_data[industry].isna()]
                    
                    # Get the amount to be allocated for this industry
                    allocation = missing_allocations[country][industry]
                    
                    # Impute each missing value
                    for region in missing_regions:
                        # Get similarity scores and calculate weights
                        similarities = similarity_matrix.loc[region, available_regions]
                        weights = similarities / similarities.sum()
                        
                        # Use weighted average of available values
                        imputed_value = (imputed_data.loc[available_regions, industry] * weights).sum()
                        imputed_data.loc[region, industry] = imputed_value
                    
                    # Scale imputed values to match allocation if positive
                    if len(missing_regions) > 0 and allocation > 0:
                        imputed_total = imputed_data.loc[missing_regions, industry].sum()
                        if imputed_total > 0:  # Avoid division by zero
                            scaling_factor = allocation / imputed_total
                            imputed_data.loc[missing_regions, industry] *= scaling_factor
        
        # Check if the solution has converged
        if (abs(imputed_data - previous_data) < tolerance).all().all():
            print(f"Converged after {iteration + 1} iterations")
            break
    
    return imputed_data

def process_sbs_datasets(
    sbs_data: Dict[str, pd.DataFrame],
    gdp_pc: pd.DataFrame
) -> Dict[str, pd.DataFrame]:
    """
    Process all SBS datasets to impute missing values.
    
    Strategy:
    1. Create similarity matrix for each dataset
    2. Impute missing values using GDP per capita similarity
    3. Verify results against country totals
    
    Args:
        sbs_data (Dict[str, pd.DataFrame]): Dictionary of SBS datasets
        gdp_pc (pd.DataFrame): GDP per capita data
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary of imputed SBS datasets
    """
    # Process each SBS dataset
    for dataset_name in ['SBSEmployment_2022', 'SBSFirms_2022', 'SBSWages_2022']:
        print(f"\nProcessing {dataset_name}...")
        
        # Create similarity matrix for current dataset
        similarity_matrix = prepare_similarity_matrix(gdp_pc, sbs_data[dataset_name])
        
        # Perform imputation
        imputed_data = impute_missing_sbs_data(
            sbs_data[dataset_name],
            similarity_matrix
        )
        
        # Update dataset with imputed values
        sbs_data[dataset_name] = imputed_data
    
        # Report results
        missing_before = sbs_data[dataset_name].isna().sum().sum()
        missing_after = imputed_data.isna().sum().sum()
        print(f"Missing values before: {missing_before}")
        print(f"Missing values after: {missing_after}")
        
        # Verify accuracy against country totals
        for country in imputed_data.index.str[:2].unique():
            country_total = imputed_data.loc[country]
            regional_sum = imputed_data[imputed_data.index.str.startswith(country) & 
                                     (imputed_data.index.str.len() == 4)].sum()
            max_diff = abs(country_total - regional_sum).max()
            print(f"{country} - Maximum difference from country total: {max_diff:.2f}")
    
    return sbs_data


In [15]:
sbs_data = {
    'SBSEmployment_2022': SBSEmployment_2022,
    'SBSFirms_2022': SBSFirms_2022, 
    'SBSWages_2022': SBSWages_2022
}
GDPpc_2022 = GDPpc_2022.rename(columns={'2022': 'GDPpc_2022'})
imputed_sbs_data = process_sbs_datasets(sbs_data, GDPpc_2022)

# Create new dataframes with imputed data
Imputed_SBSEmployment_2022 = imputed_sbs_data['SBSEmployment_2022']
Imputed_SBSFirms_2022 = imputed_sbs_data['SBSFirms_2022']
Imputed_SBSWages_2022 = imputed_sbs_data['SBSWages_2022']

print("Created imputed dataframes:")
print("- Imputed_SBSEmployment_2022")
print("- Imputed_SBSFirms_2022")
print("- Imputed_SBSWages_2022")



Processing SBSEmployment_2022...

Preparing similarity matrix...
Calculating GDP-based similarities...
Similarity matrix preparation complete

Starting SBS data imputation...

Step 1: Imputing country-level data...

Step 2: Calculating missing allocations...
Converged after 2 iterations
Missing values before: 0
Missing values after: 0
BE - Maximum difference from country total: 7873.00
BG - Maximum difference from country total: 0.00
CZ - Maximum difference from country total: 2.00
DK - Maximum difference from country total: 0.00
DE - Maximum difference from country total: 4.00
EE - Maximum difference from country total: 111411.00
IE - Maximum difference from country total: 31144.13
EL - Maximum difference from country total: 32.00
ES - Maximum difference from country total: 3.00
FR - Maximum difference from country total: 985.07
HR - Maximum difference from country total: 0.00
IT - Maximum difference from country total: 134324.40
CY - Maximum difference from country total: 75841.00
L